In [2]:
import pandas as pd 
import json

In [3]:
## https://siliconheaven.info/aws-textract-pdf-csv/
# https://aws.amazon.com/blogs/machine-learning/postprocessing-with-amazon-textract-multi-page-table-handling/

# https://github.com/aws-samples/amazon-textract-multipage-tables-processing/blob/main/Merge_Table_Notebook.ipynb
bank_statement_json=pd.read_json('./922010032985962.json')
bank_statement_json

,DocumentMetadata,JobStatus,NextToken,Blocks,ResponseMetadata
0,{'Pages': 3},SUCCEEDED,10D0z288IraBHic+a0AVGKvAjNSc+uWRpFBoH5kmvLu4ri...,"[{'BlockType': 'PAGE', 'Geometry': {'BoundingB...",{'RequestId': '4c14d901-ff64-4e29-a775-847de8d...
1,{'Pages': 3},SUCCEEDED,NaN,"[{'BlockType': 'WORD', 'Confidence': 99.995391...",{'RequestId': 'f32f1df6-6c91-417c-a8cc-78a6c9b...


In [28]:
import json
import io
import csv
with open('./922010032985962.json') as f:
      data = json.load(f)



{'DocumentMetadata': {'Pages': 3},
 'JobStatus': 'SUCCEEDED',
 'Blocks': [{'BlockType': 'WORD',
   'Confidence': 99.99539184570312,
   'Text': 'the',
   'Geometry': {'BoundingBox': {'Width': 0.018835419788956642,
     'Height': 0.008001139387488365,
     'Left': 0.37198570370674133,
     'Top': 0.8852915167808533},
    'Polygon': [{'X': 0.37198570370674133, 'Y': 0.8852915167808533},
     {'X': 0.3908211290836334, 'Y': 0.8852915167808533},
     {'X': 0.3908211290836334, 'Y': 0.8932926654815674},
     {'X': 0.37198570370674133, 'Y': 0.8932926654815674}]},
   'Id': '5de6242a-a32d-4e8a-8b67-1ce34323b5ad',
   'Page': 2},
  {'BlockType': 'WORD',
   'Confidence': 99.9808349609375,
   'Text': 'closing',
   'Geometry': {'BoundingBox': {'Width': 0.043851714581251144,
     'Height': 0.010093668475747108,
     'Left': 0.3944709599018097,
     'Top': 0.8852936029434204},
    'Polygon': [{'X': 0.3944709599018097, 'Y': 0.8852936029434204},
     {'X': 0.43832269310951233, 'Y': 0.8852936029434204},
   

In [29]:
data[1]


{'DocumentMetadata': {'Pages': 3},
 'JobStatus': 'SUCCEEDED',
 'Blocks': [{'BlockType': 'WORD',
   'Confidence': 99.99539184570312,
   'Text': 'the',
   'Geometry': {'BoundingBox': {'Width': 0.018835419788956642,
     'Height': 0.008001139387488365,
     'Left': 0.37198570370674133,
     'Top': 0.8852915167808533},
    'Polygon': [{'X': 0.37198570370674133, 'Y': 0.8852915167808533},
     {'X': 0.3908211290836334, 'Y': 0.8852915167808533},
     {'X': 0.3908211290836334, 'Y': 0.8932926654815674},
     {'X': 0.37198570370674133, 'Y': 0.8932926654815674}]},
   'Id': '5de6242a-a32d-4e8a-8b67-1ce34323b5ad',
   'Page': 2},
  {'BlockType': 'WORD',
   'Confidence': 99.9808349609375,
   'Text': 'closing',
   'Geometry': {'BoundingBox': {'Width': 0.043851714581251144,
     'Height': 0.010093668475747108,
     'Left': 0.3944709599018097,
     'Top': 0.8852936029434204},
    'Polygon': [{'X': 0.3944709599018097, 'Y': 0.8852936029434204},
     {'X': 0.43832269310951233, 'Y': 0.8852936029434204},
   

In [30]:
### Using Line rather than cell 
# def extract_tables(textract_response):
#     blocks = textract_response['Blocks']
#     blocks_map = {}
#     table_blocks = []
#     for block in blocks:
#         blocks_map[block['Id']] = block
#         if block['BlockType'] == 'PAGE' and block['Page']==2:
            
#             table_blocks.append(block)
#     return blocks_map, table_blocks

# blocks_map, table_blocks=extract_tables(data[0])
# print(type(table_blocks),len(table_blocks),table_blocks)


def extract_tables(textract_response):
    blocks = textract_response['Blocks']
    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == 'PAGE' and block['Page']==2:
            table_blocks.append(block)
    return blocks_map, table_blocks

# extract_tables(data[0])


def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    row_index=1
    column_index=1
    
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'LINE':
                    row_index = row_index+1
                    
                    col_index = column_index+1
                    
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    
                    word = blocks_map[child_id]
                    print(word)
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '
    return text


def file_tables_to_df(textract_response):
    blocks_map, table_blocks = extract_tables(textract_response)
    all_dfs = []
    if len(table_blocks) <= 0:
        return all_dfs

    for index, table_result in enumerate(table_blocks):
        data_rows = get_rows_columns_map(table_result, blocks_map)
        mem_file = io.StringIO()
        print(data_rows[1])
#         writer = csv.DictWriter(mem_file, data_rows[1])
#         for row_index in data_rows:
#             row_data = {k: v.encode('ascii', 'ignore').decode().replace('$', '') for k, v in
#                         data_rows[row_index].items()}
#             writer.writerow(row_data)
#         mem_file.seek(0)
#         df = pd.read_csv(mem_file)
#         df.to_csv(mem_file, index=False)
#         all_dfs.append(df)
#     return all_dfs

table_list=file_tables_to_df(data[1])
table_list[1]

IndexError: list index out of range